In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from fl_g13.editing.sparseSGDM import SparseSGDM
from torch.nn import CrossEntropyLoss

import flwr
from flwr.simulation import run_simulation
from fl_g13.architectures import BaseDino
from fl_g13.fl_pytorch import get_client_app, get_server_app
from fl_g13.fl_pytorch import build_fl_dependencies


print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

build_fl_dependencies() #! Remind to always put this, it will download Dino dependencies for client

2025-05-28 11:26:47.763 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: /home/massimiliano/Projects/fl-g13


Flower 1.17.0 / PyTorch 2.6.0+cu124
'vision_transformer.py' already exists.
'utils.py' already exists.


In [3]:
# Checkpoint saving settings
CHECKPOINT_DIR = "/home/massimiliano/Projects/fl-g13/checkpoints"
name = 'aron'
save_with_model_dir = False
save_every = 1

# Model hyper-parameters
head_layers=3
head_hidden_size=512
dropout_rate=0.0
unfreeze_blocks=1

# Training hyper-parameters
starting_lr = 1e-3
momentum = 0.9
weight_decay=1e-5
T_max=8
eta_min=1e-5

# Federated Training settings
batch_size = 64 # Batch size for training #! Let's stick to 64 to make training fit also on RTX 3070
local_epochs = 1 # Number of local epochs per client
number_of_rounds = 5 # Total number of federated learning rounds
fraction_fit = 1 # Fraction of clients participating in training per round
fraction_evaluate = 0.1 # Fraction of clients participating in evaluation per round
number_of_clients = 2 # Total number of clients in the simulation
min_num_clients = 2 # Minimum number of clients required for training and evaluation
partition_type = "iid" # Partitioning strategy for the dataset (e.g., "iid" or "shard")
num_shards_per_partition = 6 # Number of shards per partition (used when partition_type is "shard")
use_wandb = False # Whether to use Weights & Biases (wandb) for experiment tracking
wandb_config = None

# Device settings
device = "cuda" if torch.cuda.is_available() else "cpu"
backend_config = {
    "client_resources": {
        "num_cpus": 1, 
        "num_gpus": 0
    }
}

# When running on GPU, assign an entire GPU for each client
# Refer to Flower framework documentation for more details about Flower simulations
# and how to set up the `backend_config`
if device == "cuda":
    backend_config["client_resources"] = {
        "num_cpus": 1, 
        "num_gpus": 1
    }

print(f"Training on {device}")

Training on cuda


In [4]:
# Model
model = BaseDino(
    head_layers=head_layers, 
    head_hidden_size=head_hidden_size, 
    dropout_rate=dropout_rate, 
    unfreeze_blocks=unfreeze_blocks
    )
model.to(device)

mask = [torch.ones_like(p, device=p.device) for p in model.parameters()] # Must be done AFTER the model is moved to CUDA
optimizer = SparseSGDM(
    params=model.parameters(),
    mask=mask,
    lr=starting_lr,
    momentum=momentum,
    weight_decay=weight_decay
    )
scheduler = None
criterion = CrossEntropyLoss()

client_app = get_client_app(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    device=device,
    strategy='scheduling-lr',
    partition_type=partition_type, 
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    local_epochs=local_epochs,
    model_editing=True,
    mask_type= 'global',
    sparsity = 0.1,
)

server_app = get_server_app(
    checkpoint_dir=CHECKPOINT_DIR,
    prefix=name,
    model_class=model.__class__,
    model_config=model.get_config(), 
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    device=device, 
    save_every=save_every,
    save_with_model_dir=save_with_model_dir,
    strategy='scheduling-lr',
    num_rounds=number_of_rounds, 
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_num_clients,
    min_evaluate_clients=min_num_clients,
    min_available_clients=number_of_clients,
    use_wandb=use_wandb,
    wandb_config=wandb_config,
)

Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


⚠️ No checkpoint found at /home/massimiliano/Projects/fl-g13/checkpoints. Creating a new model.


Using cache found in /home/massimiliano/.cache/torch/hub/facebookresearch_dino_main


### Pre-train the model (head)

In [5]:
run_simulation(
    client_app=client_app,
    server_app=server_app,
    num_supernodes=number_of_clients,
    backend_config=backend_config,
)

[Server] Server on device: cuda:0
[Server] CUDA available in client: True


INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


Using strategy 'LRUpdateFedAvg'
[Server Eval Round 0] Model device: cuda:0
[Server Eval Round 0] CUDA available in server eval: True


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 13.67batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 6.5382, Metrics: {'centralized_accuracy': 0.01}
INFO :      initial parameters (loss, other metrics): 6.538191083901987, {'centralized_accuracy': 0.01}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


Entered server configure_fit()


...

Eval progress: 100%|██████████| 313/313 [00:20<00:00, 14.92batch/s]
INFO :      [Round 5] Centralized Evaluation - Loss: 0.8623, Metrics: {'centralized_accuracy': 0.7602}
INFO :      fit progress: (5, 0.8622801780415038, {'centralized_accuracy': 0.7602}, 1053.5594696509997)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)


(ClientAppActor pid=39972) [Client] Client on device: cuda:0
(ClientAppActor pid=39972) [Client] CUDA available in client: True


Eval progress: 100%|██████████| 79/79 [00:10<00:00,  7.36batch/s]


(ClientAppActor pid=39972) [Client] Client on device: cuda:0
(ClientAppActor pid=39972) [Client] CUDA available in client: True


Eval progress:  97%|█████████▋| 77/79 [00:11<00:00,  6.89batch/s]
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 1076.51s
Eval progress: 100%|██████████| 79/79 [00:11<00:00,  6.91batch/s]
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.341201981788949
INFO :      		round 2: 1.0287877602290505
INFO :      		round 3: 0.9248215903209733
INFO :      		round 4: 0.8871272100677972
INFO :      		round 5: 0.8736711945148964
INFO :      	History (loss, centralized):
INFO :      		round 0: 6.538191083901987
INFO :      		round 1: 1.3333409038214639
INFO :      		round 2: 1.012247834247522
INFO :      		round 3: 0.9111603402291624
INFO :      		round 4: 0.877304599022332
INFO :      		round 5: 0.8622801780415038
INFO :      	History (metrics, distributed, fit):
INFO :      	{'avg_drift': [(1, np.float32(0.00014970644)),
INFO :      	               (2, np.float32(9.0140966e-05)),
INFO : 